## Step 17: 메모리 관리와 순환 참조

지금까지의 DeZero는 처리 속도나 메모리 효율에 크게 신경을 쓰지 않았다. 따라서 이번 파트에서는 메모리 관리의 효율성을 높이는 방법에 대해 간단히 알아보고, 이를 구현해보도록 하겠다.

### 17.1 메모리 관리

파이썬은 기본적으로 불필요한 객체를 메모리에서 자동으로 삭제한다. 이 덕에 파이썬을 이용할 때 따로 메모리 관리를 공들여 해주지 않아도 문제 없이 작업을 수행할 수 있는 것이다.

그러나 신경망처럼 큰 데이터를 다루거나 복잡한 연산을 하는 작업에서는 종종 메모리 누수(memory leak)나 메모리 부족(out of memory)가 발생하곤 한다.

그래서 이러한 작업을 수행하려 할 때는 메모리 관리 코드를 잘 작성해주어야 한다.

먼저 파이썬이 메모리를 관리하는 방식에 대해 알아보자.

첫 번째는 참조(reference) 수를 세는 방식이고, 두 번째는 세대(generation)를 기준으로 쓸모 없어진 객체(garbage)를 회수(collection)하는 방식이다.

첫 번째 방식을 '참조 회수'. 두 번째 방법을 'Garbage Collection'이라고 한다.

### 17.2 참조 카운트 방식의 메모리 관리

파이썬 메모리 관리의 기본인 참조 카운트 방식이다. 이는 구조가 간단하고 속도가 빨라 많이 사용되는 방식으로, 객체 생성 시 0으로 초기화되는 참조 카운트가 다른 객체에 의해 참조될 때마다 +1, 그 참조가 끊길 때마다 -1씩 더하여 참조 카운트가 0이 되면 해당 객체에 대한 메모리를 회수하는 방식이다.

참조 카운트가 증가하는 경우는 다음과 같다.
- 대입 연산자 사용 시
- 함수에 인수로 전달 시
- 컨테이너 타입 객체(리스트, 튜플, 클래스 등)에 추가할 때

다음 pseudo code를 통해 과정을 이해해보도록 하자.

```python
class obj:
    pass

def f(x):
    print(x)

a = obj() # 변수에 대입: 참조 카운트 1
f(a) # 함수에 인수로 전달: 참조 카운트 2
# 함수 완료: 함수 종료 시 참조 카운트 1
a = None # 대입 해제: 참조 카운트 0 / 메모리 회수
```

먼저 obj()에 의해 생성된 객체를 a에 대입했다. 생성과 함께 대입 연산자를 통한 대입이 이루어졌으므로 이 객체의 참조 카운트는 1이 된다.

다음 줄에서 함수 f(a) 호출에 의해 참조 카운트는 다시 1 증가하여 2가 되는데, 이때 이 함수의 실행이 종료되면 인수가 반환되므로 참조 카운트는 다시 1이 된다.

마지막으로 a = None에서 참조를 끊으면 비로소 참조 카운트가 0이 되고, 이렇게 0이 되는 즉시 해당 객체는 메모리에서 삭제된다.

이처럼 참조 카운트 방식은 간단하게 동작한다. 게다가 매우 강력하기 때문에 다음과 같은 메모리 문제도 적절하게 처리할 수 있다.

```python
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c

a = b = c = None
```

a가 b를 참조하고 b가 c를 참조한다. 또한 b와 c는 각각 대입 연산이 수행되므로, a, b, c의 참조 카운트는 각각 1, 2, 2가 된다. 

이후 a = b = c = None에 의해 세 변수에 대해 모두 참조가 끊기게 되면 a의 참조 카운트가 0이 되어 메모리에서 삭제된다. 이어서 a로부터 참조를 받아 참조 카운트 1를 가지고 있던 객체 b는 a가 삭제됨에 따라 마찬가지로 참조 카운트가 0이 된다. 이는 c 변수에서도 마찬가지다.

이상이 파이썬의 참조 카운트 방식이다. 그러나 이러한 참조 카운트 방식로도 순환 참조 문재를 해결하진 못한다.

### 17.3 순환 참조

순환 참조(circular reference)를 직관적으로 이해하기 위해 다음 pseudo code를 살펴보자.

```python
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c
c.a = a

a = b = c = None
```

앞서 살펴본 코드와 거의 유사하지만, 이번에는 c에서 a로의 참조가 추가되었다. 이로 인해 세 객체 a, b, c가 서로를 참조하며 순환 구조를 그리게 되는데, 셋 중 어느 하나도 참조 카운트가 0이 되지 않기 때문에 메모리 삭제가 되지 않는다는 문제가 발생한다.

이에 따라 새로운 메모리 관리 방식인 Garbage Collection(이하 GC)이 등장한다.

GC는 객체가 생성되는 시점부터 실행되는 참조 카운트 방식과 달리, 메모리가 부족해지는 시점 혹은 사용자가 지정한 시점(gc.collect())에 파이썬 인터프리터에 의해 호출된다.

일반적인 프로그래밍 시에는 순환 참조를 특별히 의식하지 않아도 GC가 알아서 처리해준다. 하지만 메모리 해제를 미루며 GC에게만 의존하다보면 메모리 사용량이 비교적 커지게 된다.

메모리가 중요 자산 중 하나인 머신러닝 작업에서는 이러한 부분을 고려하는 것이 중요하다.

이제 이러한 부분을 DeZero에서의 문제를 해결할 수 있도록 해보자. DeZero에서는 변수 Variable과 함수 Function을 연결하는 부분에서 순환 참조가 발생한다.

<img width="80%" alt="image" src="https://user-images.githubusercontent.com/76294398/178656564-22d1d6e1-a073-4b43-b21a-5c267a3aaa02.png">

위 그림에서처럼 Function 인스턴스는 두 개의 Variable 인스턴스를 참조한다. 그리고 출력 Variable 인스턴스는 창조자인 Function 인스턴스를 참조한다.

여기서 창조자 Function 인스턴스와 출력 Variable 인스턴스 간 순환 참조가 발생한다.

### 17.4 weakref 모듈

파이썬에서는 weakref.ref 함수를 사용하여 약한 참조(weak reference)를 만들 수 있다. 약한 참조란 다른 객체를 참조하되 참조 카운트는 증가시키지 않는 기능이다.

```python
import weakref
import numpy as np

a = np.array([1,2,3])
b = weakref.ref(a)

print(b)
print(b())
```

\> <weakref at 0x10050a090; to 'numpy.ndarray' at 0x101adcbd0>

\> [1 2 3]

ndarray 인스턴스를 대상으로 실험을 진행해보았다. 먼저 a는 일반적인 방식으로 참조하고 b는 약한 참조를 갖게 했다. 이 상태로 b를 출력해보면 ndarray를 가리키는 약한 참조임을 확인할 수 있다.

여기서 a = None으로 a에 대한 참조를 끊어주면 b는 다음과 같이 출력된다.

```python
a = None
print(b)
```

\> <weakref at 0x10050a090; dead>

이와 같이 ndarray 인스턴스는 참조 카운트 방식에 따라 메모리에서 삭제된다. b도 참조를 가지고 있지만 약한 참조이기 때문에 참조 카운트에 영향을 주지 못하는 것이다. 그래서 b를 출력하면 dead라는 문자가 나오고, 이것으로 ndarray 인스턴스가 삭제됐음을 알 수 있다.

여기서 주의할 점은, 이러한 약한 참조 실험은 파이썬 인터프리터 환경에서만 통용된다는 사실이다. Ipython이나 주피터 노트북 같은 대화형 인터프리터에서는 숨은 참조를 생성하기 때문에 앞의 시나리오를 따르더라도 여전히 참조가 유지된다.

이러한 weakref 구조를 DeZero에도 적용해보도록 하자.

```python
import weakref

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])

        for output in outputs:
            output.set_creator(self)

        self.inputs = inputs
        
        # 인스턴스 변수 self.outputs가 대상을 약한 참조로 가리키도록 변경
        self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]
    ...
```

위에서 self.outputs 부분에서 출력 변수를 약한 참조하도록 변경하였다. 이에 따라 이를 참조하는 코드도 수정이 필요하다. DeZero에서는 Variable의 backward 메서드를 수정하면 된다.

```python
class Variable:
    ...
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = PriorityQueue()
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.put(f)
                seen_set.add(f)

        add_func(self.creator)

        while not funcs.empty():
            f = funcs.get()
            # 변경 전: gys = [output.grad for output in f.outputs]
            # 약한 참조로 변경된 output의 실제값에 접근하기 위해 수정
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
    ...
```

### 17.5 동작 확인

순환 참조가 없어진 DeZero에서 다음 코드를 실행해보자.

```python
for i in range(10):
    x = Variable(np.random.randn(10_000)) # 거대한 데이터
    y = square(square(square(x))) # 복잡한 계산
```

for 문을 사용하여 계산을 반복해 수행했다. 이 반복문은 다음 그림과 같은 복잡한 참조 구조를 만들어낸다.

<img width="80%" alt="image" src="https://user-images.githubusercontent.com/76294398/178683874-85455a63-2c73-4f14-88fd-cde0557790b6.png">

그리고 for 문이 두 번째 반복될 때 x와 y가 덮어 써진다. 그러면 사용자는 이전의 계산 그래프를 더 이상 참조하지 않게 된다. 참조 카운트가 0이 되므로 이 시점에 계산 그래프에 사용된 메모리가 바로 삭제된다. 이로써 DeZero의 순환 참조 문제가 해소되었다.

파이썬의 메모리 사용량은 다음과 같이 외부 라이브러리인 memory profiler 등을 이용해 측정할 수 있다.

```python
from memory_profiler import profile

@profile
# Memory Increment 확인
def test():
    for i in range(10):
        x = Variable(np.random.randn(10000))  # 거대한 데이터
        y = square(square(square(x)))  # 복잡한 계산
```

더 자세한 내용은 Tips/how_to_profile_mem_usage.md를 참고하자.